### Bokeh documentation</br> https://docs.bokeh.org/en/latest/docs/user_guide/styling/themes.html#ug-styling-themes

In [ ]:
import subprocess
import sys

def install_package(package_name):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

try:
    import bokeh
except ImportError:
    print("Bokeh is not installed. Installing...")
    install_package("bokeh")

try:
    import pandas
except ImportError:
    print("Pandas is not installed. Installing...")
    install_package("pandas")

try:
    import numpy
except ImportError:
    print("NumPy is not installed. Installing...")
    install_package("numpy")


In [ ]:
import numpy as np
import pandas as pd
# Allow Bokeh to display inside Jupyter Notebook
import bokeh.io
# A Bokeh colour pallette for the pie chart
from bokeh.palettes import Category20c
# Various Bokeh plotting and transform functionality
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.models import  Div, Spinner, TextInput
from bokeh.transform import cumsum
from math import pi
from bokeh.layouts import layout
from bokeh.tile_providers import get_provider, Vendors
# Project from latitude/longitude (EPSG:4326) into Web Mercator (EPSG:385)
from pyproj import Proj

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
# Replace 'your_json_file.json' with the path to your JSON file
json_file_path = 'Fema.json'

# Read the JSON file
with open(json_file_path, 'r') as file:
    data = pd.read_json(file,  lines=True)

# Transform the JSON data into a DataFrame
data_frame = pd.DataFrame(data)

# Print the DataFrame
print(data_frame)


In [ ]:
# Scatter plot

p = figure(
    title="Median Income vs Average House Value",
    x_axis_label="Median Income",
    y_axis_label="Average House Value"
)

p.scatter(
    # the x variable
    input_data_housing_df["MedInc"],
    # the y variable
    input_data_housing_df["AverageHouseValue"]
)
# show the visualisation
show(p)

In [ ]:
# Histogram example
# create the figure
p = figure(
    toolbar_location=None,
    title="Distribution of House Age"
)
    
# create the histogram data
hist, edges = np.histogram(
    input_data_housing_df["HouseAge"], 
    density=True, 
    bins=20
)

# display the histogram data
p.quad(
    top=hist, 
    bottom=0, 
    left=edges[:-1], 
    right=edges[1:],
    fill_color="DarkSeaGreen", 
    line_color="white"
)

# show the visualisation
show(p)

In [ ]:
# Map
projection = Proj("EPSG:3857")
mercator = projection(
    input_data_housing_df["Longitude"],
    input_data_housing_df["Latitude"]
)
source = ColumnDataSource(
    data = dict(
        x = mercator[0],
        y = mercator[1]
    )
)
south_east = projection(
    -114.131211, 
    32.534156   
)
north_west = projection(
    -124.409591,
    42.009518
)
p = figure(
    x_range=(south_east[0],north_west[0]),
    y_range=(south_east[1],north_west[1]),
    x_axis_type="mercator", 
    y_axis_type="mercator"
)
p.add_tile("OSM")
p.circle(
    x="x", 
    y="y", 
    size=2, 
    fill_color="blue", 
    fill_alpha=0.5, 
    source=source
)
show(p)

In [ ]:
# Map with hover tool-tips
projection = Proj("EPSG:3857")
mercator = projection(
    input_data_housing_df["Longitude"],
    input_data_housing_df["Latitude"]
)
source = ColumnDataSource(
    data = dict(
        x = mercator[0],
        y = mercator[1],
        aveocc = input_data_housing_df["AveOccup"],
        medinc = input_data_housing_df["MedInc"]
    )
)
south_east = projection(
    -114.131211, 
    32.534156   
)
north_west = projection(
    -124.409591,
    42.009518
)
tooltips = [
    ("Average Occupancy","@aveocc"),
    ("Median Income","@medinc")
]
p = figure(
    x_range=(south_east[0],north_west[0]),
    y_range=(south_east[1],north_west[1]),
    x_axis_type="mercator", 
    y_axis_type="mercator",
    tooltips = tooltips
)
p.add_tile("OSM")
p.circle(
    x="x", 
    y="y", 
    size=2, 
    fill_color="blue", 
    fill_alpha=0.5, 
    source=source
)
show(p)

In [ ]:
# scatter plot

import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import Div, Spinner, TextInput
from bokeh.layouts import layout

x = np.random.random(50)  # x axis 
y = np.random.random(50)  # y axis

p = figure()
points = p.circle(x, y)

div = Div(text="<p> Select Size </p>")
spinner = Spinner(title="Circle Size", low=0, high=10, step=1,
                  value=points.glyph.size, width=200)
spinner.js_link("value", points.glyph, "size")

textinput = TextInput(title="Circle Color", value=points.glyph.fill_color, width=200)
textinput.js_link("value", points.glyph, "fill_color")

layout = layout([[div, spinner], [textinput], [p]])

show(layout)


In [ ]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.models import Div, Spinner
from bokeh.layouts import layout

x = np.arange(1, 51)
y = np.random.random(50)

p = figure(x_range=(0, 51), plot_height=400, title="Column Chart")
bars = p.vbar(x=x, top=y, width=0.9, fill_color="navy")

div = Div(text="<p>Select Bar Width</p>")
spinner = Spinner(title="Bar Width", low=0.1, high=2, step=0.1,
                  value=bars.glyph.width, width=200)
spinner.js_link("value", bars.glyph, "width")

layout = layout([[div, spinner], [p]])

show(layout)

In [ ]:
import numpy as np
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from math import pi

# Create a dictionary with the category labels and their values
data = {'Category': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
        'Value': np.random.randint(1, 15, size=10)}

# Calculate the angles for the wedges based on the values
data['angle'] = [value / sum(data['Value']) * 2 * pi for value in data['Value']]

# Assign colors to the wedges
data['color'] = Category20c[len(data['Category'])]

p = figure(plot_height=400, plot_width=400, title="Pie Chart", x_range=(-1, 1), y_range=(-1, 1))

p.wedge(x=0, y=0, radius=1, start_angle=cumsum('angle', include_zero=True),
        end_angle=cumsum('angle'), line_color='white', fill_color='color', source=data)

show(p)


In [ ]:
import numpy as np
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.models import ColumnDataSource, LabelSet
from math import pi

# Create a dictionary with the category labels and their values
data = {'Category': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
        'Value': np.random.randint(1, 15, size=10)}

# Calculate the angles for the wedges based on the values
data['angle'] = [value / sum(data['Value']) * 2 * pi for value in data['Value']]

# Assign colors to the wedges
data['color'] = Category20c[len(data['Category'])]

p = figure(plot_height=400, plot_width=400, title="Pie Chart", x_range=(-1.2, 1.2), y_range=(-1.2, 1.2))

p.wedge(x=0, y=0, radius=1, start_angle=cumsum('angle', include_zero=True),
        end_angle=cumsum('angle'), line_color='white', fill_color='color', source=data)

# Add labels to the pie chart
data['percentage'] = [f"{value / sum(data['Value']) * 100:.1f}%" for value in data['Value']]
data['cumulative_angle'] = [(start + end) / 2 for start, end in zip(cumsum(data['angle'], include_zero=True), cumsum(data['angle']))]

source_labels = ColumnDataSource(data)
labels = LabelSet(x='x', y='y', text='percentage', angle='cumulative_angle', source=source_labels,
                  render_mode='canvas', text_font_size='10pt', text_color='white')

# Calculate the x and y coordinates for the labels
source_labels.data['x'] = 0.8 * np.cos(np.array(source_labels.data['cumulative_angle']) - pi / 2)
source_labels.data['y'] = 0.8 * np.sin(np.array(source_labels.data['cumulative_angle']) - pi / 2)

p.add_layout(labels)

show(p)


In [76]:
from bokeh.models import ColumnDataSource, GMapOptions
from bokeh.plotting import gmap, show

map_options = GMapOptions(lat=30.2861, lng=-97.7394, map_type="roadmap", zoom=11)

# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:
p = gmap("GOOGLE_API_KEY", map_options, title="Austin")

source = ColumnDataSource(
    data=dict(lat=[ 30.29,  30.20,  30.29],
              lon=[-97.70, -97.74, -97.78])
)

p.circle(x="lon", y="lat", size=15, fill_color="blue", fill_alpha=0.8, source=source)

show(p)